In [1]:
import sys
sys.path.append("models/")

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import pickle as pkl
import numpy as np
from time import time

import glob
import itertools

from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate, GroupKFold
import statsmodels.api as sm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from M1_util_train_test import load_model, test
import mnl
import linear_reg
from dataloader import SurveyDataset, load_aggregate_travel_behavior, load_demo_v1, train_test_split_data
from setup import out_dir, data_dir, image_dir, model_dir, proj_dir



# Load Model Embeddings

In [2]:
data_version = '1571'

model_type = 'AE'
sampling = 's'

zoomlevel = 'zoom15'
output_dim = 3
model_run_date = '2208'
v2 = 1

variable_names = ['active','auto','mas','pt', 'trpgen']

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

In [3]:
with open(proj_dir+"latent_space/"+model_type+"_"+zoomlevel+"_"+str(output_dim**2*2048)+
                       "_"+str(v2)+"_"+model_run_date+".pkl", "rb") as f: 
    encoder_output = pkl.load(f)
    im = pkl.load(f)
    ct = pkl.load(f)

In [4]:
# Aggregate Embeddings
unique_ct = list(set(ct))
unique_ct.sort()
ct = np.array(ct)
embed_ae = []
for i in unique_ct:
    embed_ae.append(np.mean(encoder_output[ct == i], axis=0))
embed_ae = np.array(embed_ae)

In [5]:
data_version = '1571'

model_type = 'SAE'
sampling = 's'

zoomlevel = 'zoom15'
output_dim = 3
model_run_date = '2208'

v1 = 'F'
v2 = 1


variable_names = ['active','auto','mas','pt', 'trpgen']

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

with open(proj_dir+"latent_space/"+model_type+"_"+zoomlevel+"_"+str(output_dim**2*2048)+"_"+
                       v1+"_"+str(v2)+"_"+model_run_date+".pkl", "rb") as f: 
    encoder_output = pkl.load(f)
    im = pkl.load(f)
    ct = pkl.load(f)
    
# Aggregate Embeddings
unique_ct = list(set(ct))
unique_ct.sort()
ct = np.array(ct)
embed_sae = []
for i in unique_ct:
    embed_sae.append(np.mean(encoder_output[ct == i], axis=0))
embed_sae = np.array(embed_sae)

In [6]:
model_type = 'SSD'
zoomlevel = 'zoom15'
output_dim = 3
model_run_date = '2208'

with open(proj_dir+"latent_space/SSD_"+zoomlevel+"_"+str(output_dim**2*2048)+"_"+str(v2)+"_"+
                       str(model_run_date)+".pkl", "rb") as f:
    encoder_output = pkl.load(f)
    im = pkl.load(f)
    ct = pkl.load(f)
    
# Aggregate Embeddings
unique_ct = list(set(ct))
unique_ct.sort()
ct = np.array(ct)
embed_ssd = []
for i in unique_ct:
    embed_ssd.append(np.mean(encoder_output[ct == i], axis=0))
embed_ssd = np.array(embed_ssd)

# Load Demographics

In [25]:
# demographic variables

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

demo_cs, demo_np = load_demo_v1(data_dir,norm='minmax')
demo = np.hstack((np.array(demo_cs).reshape(-1,1), demo_np))
demo = pd.DataFrame(demo, columns = ['geoid'] + demo_variables)
demo = train_test_split_data(demo, data_version)

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone', #'pct_nonwhite',
         'pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

x_demo = demo[demo_variables].to_numpy(dtype=float)

groups = demo['train_test']

group_split = GroupKFold(n_splits=5)

# Load Trip Behavior

In [8]:
file = "origin_trip_behavior.csv"
df_pivot = load_aggregate_travel_behavior(file, data_version)

y = df_pivot[variable_names].to_numpy()[:,:4]

In [26]:
x = np.concatenate([x_demo, embed_ssd, embed_ae, embed_sae], axis=1)


In [28]:
dim_demo = demo.shape[1]
dim_embed_ae = embed_ae.shape[1]
dim_embed_sae = embed_sae.shape[1]
dim_embed_ssd = embed_ssd.shape[1]


# 1. Linear Regression

## 1.1 Auto

In [29]:
y_index = 1

In [30]:
# Lasso
for a in (1e-4)*np.array([0.1,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.491e-01, tolerance: 6.768e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.989e-01, tolerance: 6.596e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-05 	 Train score: 0.9994 	 Cross val score: 0.4900 	 Nonzero coef: 3557


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.642e+00, tolerance: 6.768e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.342e+00, tolerance: 6.596e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-04 	 Train score: 0.9616 	 Cross val score: 0.5122 	 Nonzero coef: 1029


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.204e+00, tolerance: 6.768e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.930e-01, tolerance: 6.596e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

...

Parameter: 8.00e-04 	 Train score: 0.6946 	 Cross val score: 0.6199 	 Nonzero coef: 107


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.258e-01, tolerance: 6.768e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.393e-01, tolerance: 6.596e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-03 	 Train score: 0.6778 	 Cross val score: 0.6185 	 Nonzero coef: 88


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.629e-02, tolerance: 6.769e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.267e-02, tolerance: 7.111e-03
  model = cd_fast.enet_coordinate_descent(


Parameter: 2.00e-03 	 Train score: 0.6435 	 Cross val score: 0.6050 	 Nonzero coef: 46
Parameter: 5.00e-03 	 Train score: 0.5973 	 Cross val score: 0.5725 	 Nonzero coef: 22


## 1.2 PT 

In [13]:
y_index = 3

In [14]:
# Lasso
for a in (1e-4)*np.array([0.1,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.979e-01, tolerance: 1.168e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.792e-01, tolerance: 1.130e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-05 	 Train score: 0.9972 	 Cross val score: 0.3509 	 Nonzero coef: 2308


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.778e+00, tolerance: 1.168e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.641e+00, tolerance: 1.130e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-04 	 Train score: 0.8709 	 Cross val score: 0.4096 	 Nonzero coef: 588


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.381e+00, tolerance: 1.168e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.189e+00, tolerance: 1.130e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

...

Parameter: 8.00e-04 	 Train score: 0.5473 	 Cross val score: 0.4722 	 Nonzero coef: 50


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.726e+00, tolerance: 1.168e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.910e+00, tolerance: 1.130e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-03 	 Train score: 0.5298 	 Cross val score: 0.4641 	 Nonzero coef: 40


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.184e+00, tolerance: 1.281e-03
  model = cd_fast.enet_coordinate_descent(


Parameter: 2.00e-03 	 Train score: 0.4870 	 Cross val score: 0.4452 	 Nonzero coef: 20
Parameter: 5.00e-03 	 Train score: 0.4437 	 Cross val score: 0.4248 	 Nonzero coef: 11


## 1.3 Active

In [17]:
y_index = 0

In [18]:
# Lasso
for a in (1e-4)*np.array([0.1,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.537e-01, tolerance: 3.383e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.416e-01, tolerance: 3.240e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-05 	 Train score: 0.9988 	 Cross val score: 0.3209 	 Nonzero coef: 3158


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.760e+00, tolerance: 3.383e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.602e+00, tolerance: 3.240e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-04 	 Train score: 0.9328 	 Cross val score: 0.3646 	 Nonzero coef: 897


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.660e+00, tolerance: 3.383e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.297e+00, tolerance: 3.240e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

...

Parameter: 8.00e-04 	 Train score: 0.5768 	 Cross val score: 0.4922 	 Nonzero coef: 76


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.423e+00, tolerance: 3.383e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.834e+00, tolerance: 3.240e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 1.00e-03 	 Train score: 0.5597 	 Cross val score: 0.4943 	 Nonzero coef: 63


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.072e+00, tolerance: 3.383e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.453e+00, tolerance: 3.240e-03
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

Parameter: 2.00e-03 	 Train score: 0.5209 	 Cross val score: 0.4800 	 Nonzero coef: 34
Parameter: 5.00e-03 	 Train score: 0.4738 	 Cross val score: 0.4494 	 Nonzero coef: 18


# 2. MNL

In [31]:
from util_aggregate_models import mnl_torch

# dataloader and model definition

lr_list = [0.0001, 0.0002]
wd_list = [0.1, 0.01, 0.001, 0.0001]

results = {}
for i in range(5):
    
    train_filter = groups != i
    test_filter = groups == i
    x_train = x[train_filter]
    y_train = y[train_filter]
    x_test = x[test_filter]
    y_test = y[test_filter]
    sst_train = np.sum(np.power(y_train - np.mean(y_train, axis=0), 2), axis=0)
    sst_test = np.sum(np.power(y_test - np.mean(y_test, axis=0), 2), axis=0)
    
    trainset = SurveyDataset(torch.tensor(x_train,  dtype=torch.float), torch.tensor(y_train, dtype=torch.float))
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

    testset = SurveyDataset(torch.tensor(x_test, dtype=torch.float), torch.tensor(y_test, dtype=torch.float))
    testloader = DataLoader(testset, batch_size=len(testset), shuffle=False)

    ret_dict = mnl_torch(trainloader, testloader, x_train.shape[-1], sst_train, sst_test, lr_list=lr_list, wd_list=wd_list)
    
    results[i] = ret_dict
    

[lr: 1.00e-04, wd: 1.00e-01]
[epoch: 995] Train KL loss: 0.104 Train R2 score: 0.659 0.730 0.119 0.652 
[epoch: 995] Test KL loss: 0.135 Test R2 score: 0.482 0.623 -0.026 0.522 

[lr: 1.00e-04, wd: 1.00e-02]
Diverging. stop.
[epoch:  50] Train KL loss: 0.140 Train R2 score: 0.462 0.571 -0.006 0.432 
[epoch:  50] Test KL loss: 0.142 Test R2 score: 0.450 0.594 -0.019 0.479 

[lr: 1.00e-04, wd: 1.00e-03]
Diverging. stop.
[epoch: 940] Train KL loss: 0.100 Train R2 score: 0.679 0.751 0.157 0.672 
[epoch: 940] Test KL loss: 0.136 Test R2 score: 0.472 0.621 -0.031 0.516 

[lr: 2.00e-04, wd: 1.00e-02]
Early stopping at epoch 975
[epoch: 970] Train KL loss: 0.098 Train R2 score: 0.683 0.754 0.166 0.681 
[epoch: 970] Test KL loss: 0.136 Test R2 score: 0.474 0.619 -0.030 0.515 

[lr: 2.00e-04, wd: 1.00e-03]
Diverging. stop.
[epoch: 885] Train KL loss: 0.100 Train R2 score: 0.669 0.745 0.166 0.673 
[epoch: 885] Test KL loss: 0.136 Test R2 score: 0.470 0.621 -0.031 0.525 

[lr: 2.00e-04, wd: 1.00e-

In [32]:
df = []

for (lr, wd) in itertools.product(lr_list, wd_list):

    new = []
    
    for i in range(5):
        new2 = []
        for k,v in results[i][(lr,wd)].items():
            new2.append(results[i][(lr,wd)][k]) 
        new.append(new2)
        
    new = np.array(new) 
    
    df.append([lr] + [wd] + list(np.mean(new, axis=0)))


pd.DataFrame(np.array(df), columns = ['learning_rate','weight_decay','train_kl_loss','test_kl_loss','train_r2_auto','train_r2_active','train_r2_pt',
                                     'test_r2_auto','test_r2_active','test_r2_pt']).sort_values(by='test_kl_loss')

,learning_rate,weight_decay,train_kl_loss,test_kl_loss,train_r2_auto,train_r2_active,train_r2_pt,test_r2_auto,test_r2_active,test_r2_pt
0,0.0001,0.1000,0.102800,0.134376,0.737153,0.661880,0.664889,0.617541,0.487780,0.488384
2,0.0001,0.0010,0.103304,0.134625,0.731991,0.654891,0.663010,0.616458,0.485706,0.486973
1,0.0001,0.0100,0.118027,0.138307,0.670948,0.572856,0.580822,0.593521,0.472236,0.438084
3,0.0001,0.0001,0.118884,0.139230,0.666385,0.574423,0.572643,0.595334,0.471600,0.447548
6,0.0002,0.0010,0.127245,0.146126,0.647713,0.551177,0.546745,0.570847,0.448504,0.374795
7,0.0002,0.0001,0.141861,0.149379,0.578649,0.460263,0.435881,0.568185,0.435760,0.364482
5,0.0002,0.0100,0.128211,0.150128,0.642682,0.529026,0.500786,0.576009,0.406622,0.038471
4,0.0002,0.1000,0.139445,0.158998,0.591329,0.486756,0.448766,0.545088,0.431036,0.231922
